In [1]:
import numpy as np
import pandas as pd

pd.options.display.max_rows = 20
# Configura pandas para mostrar un
# máximo de 20 filas al imprimir DataFrames. 
# Esto ayuda a mantener la salida manejable en la consola.

pd.options.display.max_colwidth = 80
# Configura pandas para que la anchura máxima 
# de las columnas sea de 80 caracteres al 
# mostrar datos. Esto es útil para asegurarse 
# de que las columnas no se trunquen demasiado en la visualización.

pd.options.display.max_columns = 20
# Configura pandas para mostrar un máximo
# de 20 columnas al imprimir DataFrames. 
# Similar a max_rows, esto ayuda a 
# controlar la cantidad de datos mostrados.

np.random.seed(12345)


import matplotlib.pyplot as plt

plt.rc("figure", figsize=(10, 6)) # Configura matplotlib para que todas
# las figuras tengan un tamaño predeterminado de 10 pulgadas de ancho 
# por 6 pulgadas de alto. rc es una función de matplotlib que se 
# utiliza para configurar parámetros globales.

np.set_printoptions(precision=4, suppress=True)
# Configura numpy para mostrar los números con una 
# precisión de 4 decimales y suprimir el uso de 
# notación científica (suppress=True) en la salida impresa.

# 3. Tratamiento de datos (Wrangling): Unir (Join) ,combinar (Combine) y remodelar (Reshape)

En muchas aplicaciones, los datos pueden estar dispersos en varios archivos o bases de datos, o estar organizados de una forma que no es conveniente analizar. 

En primer lugar, se introduce el concepto de indexación jerárquica `(Hierarchical Indexing)`en pandas, que se utiliza ampliamente en algunas de estas operaciones.

 ## 3.1 Indexación jerárquica

La indexación jerárquica es una característica importante de pandas que le permite tener múltiples (dos o más) niveles de índice en un eje. Otra forma de pensar en ello es que proporciona una manera para que usted pueda trabajar con datos de mayor dimensión en una forma de menor dimensión. Empecemos con un ejemplo sencillo: crear una Serie con una lista de listas (o arrays) como índice:

In [2]:
data = pd.Series(np.random.uniform(size=9),
        index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.929616
   2    0.316376
   3    0.183919
b  1    0.204560
   3    0.567725
c  1    0.595545
   2    0.964515
d  2    0.653177
   3    0.748907
dtype: float64

Lo que está viendo es una vista de una Serie con un Multiíndice (`MultiIndex`) como índice. Los "huecos" en la visualización del índice significan "use la etiqueta directamente arriba":

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

Con un objeto indexado jerárquicamente, es posible la llamada indexación parcial, que permite seleccionar de forma concisa subconjuntos de los datos:

In [5]:
data["b"]

1    0.204560
3    0.567725
dtype: float64

In [6]:
data["b":"c"]

b  1    0.204560
   3    0.567725
c  1    0.595545
   2    0.964515
dtype: float64

In [10]:
data.loc[["b", "c"]]

b  1    0.204560
   3    0.567725
c  1    0.595545
   2    0.964515
dtype: float64

La selección es posible incluso desde un nivel "interior". Aquí seleccionoamos todos los valores que tienen el valor 2 del segundo nivel de índice:

In [11]:
data.loc[:, 2]

a    0.316376
c    0.964515
d    0.653177
dtype: float64

### **Ejemplo 3.1**

In [14]:
data_1 = {
    'Store': ['Store1', 'Store1', 'Store1', 'Store2', 'Store2', 'Store2'],
    'Product': ['A', 'A', 'B', 'A', 'B', 'B'],
    'Date': ['2024-07-01', '2024-07-02', '2024-07-01', '2024-07-01', '2024-07-02', '2024-07-03'],
    'Sales': [100, 150, 200, 300, 400, 500]
}
df = pd.DataFrame(data_1)

df['Date'] = pd.to_datetime(df['Date'])  # Convertir la columna 'Date' a tipo datetime
df

,Store,Product,Date,Sales
0,Store1,A,2024-07-01,100
1,Store1,A,2024-07-02,150
2,Store1,B,2024-07-01,200
3,Store2,A,2024-07-01,300
4,Store2,B,2024-07-02,400
5,Store2,B,2024-07-03,500


**Configurar a Multiindex**

In [16]:
df.set_index(['Store', 'Product', 'Date'], inplace=True)
df

Sales
Store  Product Date             
Store1 A       2024-07-01    100
               2024-07-02    150
       B       2024-07-01    200
Store2 A       2024-07-01    300
       B       2024-07-02    400
               2024-07-03    500

La indexación jerárquica desempeña un papel importante en la reorganización de los datos y en las operaciones basadas en grupos, como la formación de una tabla dinámica. Por ejemplo, puede reorganizar estos datos en un DataFrame utilizando su método `unstack` ('desapilar'):

Volvamos al dataframe data

In [17]:
data

a  1    0.929616
   2    0.316376
   3    0.183919
b  1    0.204560
   3    0.567725
c  1    0.595545
   2    0.964515
d  2    0.653177
   3    0.748907
dtype: float64

In [18]:
data.unstack()

,1,2,3
a,0.929616,0.316376,0.183919
b,0.204560,NaN,0.567725
c,0.595545,0.964515,NaN
d,NaN,0.653177,0.748907


La operación inversa de desapilar (`unstack`) es apilar (`stack`):

In [19]:
data.unstack().stack()

a  1    0.929616
   2    0.316376
   3    0.183919
b  1    0.204560
   3    0.567725
c  1    0.595545
   2    0.964515
d  2    0.653177
   3    0.748907
dtype: float64

### Vamos a desapilar el dataframe del ejemplo 3.1

In [22]:
df

Sales
Store  Product Date             
Store1 A       2024-07-01    100
               2024-07-02    150
       B       2024-07-01    200
Store2 A       2024-07-01    300
       B       2024-07-02    400
               2024-07-03    500

In [24]:
df.unstack()

Sales                      
Date           2024-07-01 2024-07-02 2024-07-03
Store  Product                                 
Store1 A            100.0      150.0        NaN
       B            200.0        NaN        NaN
Store2 A            300.0        NaN        NaN
       B              NaN      400.0      500.0

In [26]:
df.unstack('Store')

Sales       
Store              Store1 Store2
Product Date                    
A       2024-07-01  100.0  300.0
        2024-07-02  150.0    NaN
B       2024-07-01  200.0    NaN
        2024-07-02    NaN  400.0
        2024-07-03    NaN  500.0

In [27]:
df.unstack('Product')

Sales       
Product                A      B
Store  Date                    
Store1 2024-07-01  100.0  200.0
       2024-07-02  150.0    NaN
Store2 2024-07-01  300.0    NaN
       2024-07-02    NaN  400.0
       2024-07-03    NaN  500.0

#### Con un DataFrame, cualquiera de los ejes puede tener un índice jerárquico:

In [62]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                     ["Green", "Red", "Green"]])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

Los niveles jerárquicos pueden tener nombres (como cadenas o cualquier objeto Python). Si es así, aparecerán en la salida de la consola:

In [63]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

Estos nombres sustituyen al atributo name, que sólo se utiliza con índices de un solo nivel. Puedes ver cuántos niveles tiene un índice accediendo a su atributo `nlevels`: 



In [38]:
frame.index.nlevels

2

In [39]:
frame.columns.nlevels

2

Con la indexación parcial de columnas puede seleccionar grupos de columnas de forma similar:

In [64]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

Un `MultiIndex` puede crearse por sí mismo y luego reutilizarse; las columnas del DataFrame anterior con nombres de nivel también podrían crearse así:

In [65]:
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"],
                          ["Green", "Red", "Green"]],
                          names=["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

## Reordenación y clasificación de niveles

A veces puede ser necesario reorganizar el orden de los niveles en un eje u ordenar los datos por los valores de un nivel específico. El método `swaplevel` toma dos números o nombres de nivel y devuelve un nuevo objeto con los niveles intercambiados (pero los datos permanecen inalterados):

In [66]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [67]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

`sort_index` ordena por defecto los datos lexicográficamente utilizando todos los niveles del índice, pero puede elegir utilizar sólo un nivel o un subconjunto de niveles para ordenar pasando el argumento `level`. Por ejemplo:

In [73]:
frame.sort_index()

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [72]:
frame.sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [74]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [75]:
frame.swaplevel(0, 1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [76]:
frame.swaplevel(0, 1).sort_index(level=0)
#

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

> El rendimiento de la selección de datos es mucho mejor en objetos indexados jerárquicamente si el índice está ordenado lexicográficamente empezando por el nivel más externo, es decir, el resultado de llamar a `sort_index(level=0)` o `sort_index()`.

## Resumen estadístico por niveles

Muchas estadísticas descriptivas y de resumen en DataFrame y Series tienen una opción de nivel en la que puede especificar el nivel por el que desea agregar en un eje concreto. Considere el DataFrame anterior; podemos agregar por nivel en las filas o columnas, así:

In [77]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [79]:
frame.groupby(level="color", axis="columns").sum()

C:\Users\juanj\AppData\Local\Temp\ipykernel_4144\775557097.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  frame.groupby(level="color", axis="columns").sum()


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## Indexación con las columnas de un DataFrame

No es inusual querer utilizar una o más columnas de un DataFrame como índice de fila; alternativamente, puede desear mover el índice de fila a las columnas del DataFrame. He aquí un ejemplo de DataFrame:

In [80]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


La función `set_index` de DataFrame creará un nuevo DataFrame utilizando una o más de sus columnas como índice:

In [81]:
frame2 = frame.set_index(["c", "d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

Por defecto, las columnas se eliminan del DataFrame, aunque puede dejarlas pasando `drop=False` a `set_index`:

In [82]:
frame.set_index(["c", "d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

`reset_index`, por otro lado, hace lo contrario que set_index; los niveles de índice jerárquico se mueven a las columnas:

In [83]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 3.2 Combinar y fusionar conjuntos de datos

Los datos contenidos en los objetos pandas pueden combinarse de varias maneras:

`pandas.merge` : Conectar filas en DataFrames basándose en una o más claves. Esto resultará familiar a los usuarios de SQL u otras bases de datos relacionales, ya que implementa operaciones de unión de bases de datos.

`pandas.concat`: Concatena o "apila" objetos a lo largo de un eje.

`combine_first`: Empalma (Splice) datos superpuestos para rellenar los valores que faltan en un objeto con valores de otro.

## Uniones de DataFrames al estilo de las bases de datos

Las operaciones de `Merge` o `join` combinan conjuntos de datos enlazando filas mediante una o varias claves. Estas operaciones son particularmente importantes en bases de datos relacionales (ejm SQL). La función `pandas.merge` en pandas es el principal punto de entrada para utilizar estos algoritmos en sus datos. Empecemos con un ejemplo sencillo:

In [84]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [85]:
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})

df2

,key,data2
0,a,0
1,b,1
2,d,2


Este es un ejemplo de un join muchos-a-uno; los datos en df1 tienen múltiples filas etiquetadas como a y b, mientras que df2 tiene sólo una fila para cada valor en la columna clave. Llamando a `pandas.merge` con estos objetos, obtenemos:

In [87]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


Nótese que no se ha especificado en qué columna unir. Si no se especifica esa información, `pandas.merge` utiliza los nombres de las columnas solapadas como claves. Sin embargo, es una buena práctica especificarlo explícitamente:

In [86]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


En general, el orden de salida de las columnas en las operaciones `pandas.merge` no está especificado.

Si los nombres de las columnas son diferentes en cada objeto, puede especificarlos por separado:

In [88]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})

df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [89]:
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [30]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


Puede notar que los valores "c" y "d" y los datos asociados faltan en el resultado. Por defecto, `pandas.merge` hace una unión "inner" (interna); las claves en el resultado son la intersección, o el conjunto común encontrado en ambas tablas. Otras opciones posibles son "left", "right" y "outer". La unión externa toma la unión de las claves, combinando el efecto de aplicar las uniones izquierda y derecha:

In [90]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,c,3,<NA>
7,d,<NA>,2


In [32]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


En una unión externa (outer join), las filas de los objetos DataFrame izquierdo o derecho que no coincidan en las claves del otro DataFrame aparecerán con valores NA en las columnas del otro DataFrame para las filas que no coincidan.

`how=inner` :  Utiliza sólo las combinaciones de claves observadas en ambas tablas

`how="left"`: Utiliza todas las combinaciones de claves de la tabla de la izquierda

`how="right"`: Utiliza todas las combinaciones de claves de la tabla de la derecha

`how="outer"` : Utilizar conjuntamente todas las combinaciones de claves observadas en ambas tablas

Las fusiones (merges) de muchos a muchos forman el producto cartesiano de las claves coincidentes. He aquí un ejemplo:

In [100]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                    "data1": pd.Series(range(6), dtype="Int64")})

df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [101]:
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                    "data2": pd.Series(range(5), dtype="Int64")})
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [102]:
pd.merge(df1, df2, on="key", how="left")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


Como había tres filas "b" en el DataFrame izquierdo y dos en el derecho, hay seis filas "b" en el resultado. El método join pasado al argumento de la palabra clave `how` afecta sólo a los valores clave distintos que aparecen en el resultado:

In [103]:
pd.merge(df1, df2, how="inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,4,0
7,a,4,2
8,b,5,1
9,b,5,3


Para combinar (merge) con varias claves, pase una lista de nombres de columnas:

In [104]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [105]:
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [106]:
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,bar,one,3,6
1,bar,two,<NA>,7
2,foo,one,1,4
3,foo,one,1,5
4,foo,two,2,<NA>


Una última cuestión a tener en cuenta en las operaciones de fusión (merge) es el tratamiento de los nombres de columnas que se solapan. Por ejemplo:

In [107]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


Mientras que puede tratar el solapamiento manualmente, `pandas.merge` tiene una opción de `suffixes` para especificar cadenas a añadir a los nombres solapados en los objetos DataFrame izquierdo y derecho:

In [108]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


Observar la siguiente tabla para una referencia de argumentos en `pandas.merge`. La siguiente sección cubre la unión (join) usando el índice de filas del DataFrame.

`left` : DataFrame a fusionar en el lado izquierdo.

`right` : DataFrame que se fusionará en el lado derecho.

`how`: Tipo de join a aplicar: "inner", "outer", "left", o "right"; por defecto es "inner".

`on`: Nombres de las columnas a unir (join). Deben encontrarse en ambos objetos DataFrame. Si no se especifica y no se dan otras claves de unión (join), se utilizará la intersección de los nombres de columna de la izquierda y la derecha como claves de join.

`left_on`: Columnas del DataFrame izquierdo (left) que se utilizarán como claves de unión (join). Puede ser un único nombre de columna o una lista de nombres de columna.

`right_on`: Análogo a left_on para DataFrame derecho(right).

`left_index`: Utiliza el índice de la fila de la izquierda como clave de unión(join)  (o claves, si es un MultiIndex).

`right_index`: Análogo a `left_index`

`sort`:	Ordena los datos fusionados lexicográficamente por las claves de unión (join); Falso por defecto. 

`suffixes`: Tuple de valores de cadena para añadir a los nombres de columna en caso de solapamiento; por defecto ("_x", "_y") (por ejemplo, si "data" está en ambos objetos DataFrame, aparecerá como "data_x" y "data_y" en el resultado).

`copy`: Si es False, evita copiar datos en la estructura de datos resultante en algunos casos excepcionales; por defecto siempre copia.

`validate`: 	Verifica si la fusión (merge) es del tipo especificado, ya sea uno a uno, uno a muchos o muchos a muchos. Consulte el docstring para obtener más información sobre las opciones.

`indicator`: 	Añade una columna especial `_merge` que indica el origen de cada fila; los valores serán "left_only", "right_only" o "both" en función del origen de los datos unidos (joined) en cada fila.

## Merging en índice (Index)

En algunos casos, la(s) clave(s) de fusión(merging) de un DataFrame se encuentra(n) en su índice (etiquetas de fila). En este caso, puede pasar `left_index=True` o `right_index=True` (o ambos) para indicar que el índice debe utilizarse como clave de fusión:

In [109]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [110]:
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])
right1

,group_val
a,3.5
b,7.0


In [44]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


> Si observa detenidamente aquí, verá que se han conservado los valores de índice de left1, mientras que en otros ejemplos anteriores, se eliminan los índices de los objetos DataFrame de entrada. Dado que el índice de right1 es único, esta fusión "muchos-a-uno" (con el método por defecto how="inner") puede preservar los valores de índice de left1 que corresponden a filas en la salida.

Dado que el método de fusión (merge) por defecto es intersecar las claves de unión (join) , puede formar la unión de ellas con una unión (join) externa:

In [45]:
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


Con datos indexados jerárquicamente, las cosas son más complicadas, ya que la unión (join) sobre índice equivale a una fusión (merge) de varias claves:

In [111]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                               "Nevada", "Nevada"],
                      "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": pd.Series(range(5), dtype="Int64")})
lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [112]:
righth_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)

In [113]:
righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64",
                                index=righth_index), 
                        "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64",
                                             index=righth_index)})
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

En este caso, debe indicar varias columnas para fusionar en forma de lista (tenga en cuenta el tratamiento de los valores de índice duplicados con `how="outer"`):

In [49]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)


,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [50]:
pd.merge(lefth, righth, left_on=["key1", "key2"],
         right_index=True, how="outer")

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
4,Nevada,2000,<NA>,2,3


También es posible utilizar los índices de ambos lados de la fusión:

In [51]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"]).astype("Int64")
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [52]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=["b", "c", "d", "e"],
                      columns=["Missouri", "Alabama"]).astype("Int64")
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [53]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


DataFrame tiene un método de instancia `join` para simplificar la combinación por índice. También se puede utilizar para combinar varios objetos DataFrame que tengan índices iguales o similares pero columnas que no se solapen. En el ejemplo anterior, podríamos haber escrito:

In [54]:
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


Comparado con `pandas.merge`, el método `join` de DataFrame realiza una unión a la izquierda en las claves de unión por defecto. También soporta unir el índice del DataFrame pasado en una de las columnas del DataFrame al que se está llamando:

In [55]:
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


Puede pensar en este método como una unión de datos "dentro" del objeto cuyo método join fue llamado.

Por último, para simples fusiones (merge) índice sobre índice, puede pasar una lista de DataFrames a unir como alternativa al uso de la función más general `pandas.concat` descrita en la siguiente sección:



In [56]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=["a", "c", "e", "f"],
                       columns=["New York", "Oregon"])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [57]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


In [58]:
left2.join([right2, another], how="outer")

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
b,<NA>,<NA>,7,8,NaN,NaN
d,<NA>,<NA>,11,12,NaN,NaN
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0


### Martes 16/07 /2024 Concatenar a lo largo de un eje 

Otro tipo de operación de combinación de datos se denomina indistintamente concatenación o apilamiento (stacking). La función concatenar de NumPy puede hacer esto con matrices NumPy:



In [59]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [60]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

En el contexto de objetos pandas como Series y DataFrame, tener ejes etiquetados permite generalizar aún más la concatenación de arrays. En particular, usted tiene un número de preocupaciones adicionales:

- Si los objetos están indexados de forma diferente en los otros ejes, ¿debemos combinar los elementos distintos en estos ejes o utilizar sólo los valores en común?

- ¿Es necesario que los trozos (chunk) de datos concatenados sean identificables como tales en el objeto resultante?

- ¿Contiene el "eje de concatenación" datos que deban conservarse? En muchos casos, es mejor descartar las etiquetas enteras por defecto de un DataFrame durante la concatenación.

La función `concat` en pandas proporciona una forma consistente de abordar cada una de estas cuestiones. Daré una serie de ejemplos para ilustrar cómo funciona. Supongamos que tenemos tres Series sin solapamiento de índices (index overlap):

In [61]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

Al llamar a `pandas.concat` con estos objetos en una lista se pegan los valores y los índices:

In [62]:
s1

a    0
b    1
dtype: Int64

In [63]:
s2

c    2
d    3
e    4
dtype: Int64

In [64]:
s3

f    5
g    6
dtype: Int64

In [65]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

Por defecto, `pandas.concat` funciona con `axis="index"`, produciendo otra Serie. Si pasa `axis="columns"`, el resultado será un DataFrame:

In [66]:
pd.concat([s1, s2, s3], axis="columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


En este caso no hay solapamiento en el otro eje, que como puede ver es la unión (el join "externo") de los índices. En su lugar, puede intersecarlos pasando join="inner":

In [67]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: Int64

In [68]:
pd.concat([s1, s4], axis="columns")

,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


In [69]:
pd.concat([s1, s4], axis="columns", join="inner")

,0,1
a,0,0
b,1,1


En este último ejemplo, las etiquetas "f" y "g" han desaparecido debido a la opción `join="inner"`.

Un problema potencial es que las piezas concatenadas no son identificables en el resultado. Supongamos, en cambio, que desea crear un índice jerárquico en el eje de concatenación. Para ello, utilice el argumento keys:

In [70]:
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: Int64

In [71]:
result.unstack()

,a,b,f,g
one,0,1,<NA>,<NA>
two,0,1,<NA>,<NA>
three,<NA>,<NA>,5,6


En el caso de combinar Series a lo largo de axis="columns", las claves se convierten en las cabeceras de las columnas del DataFrame:

In [72]:
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


La misma lógica se extiende a los objetos DataFrame:

In [73]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"],
                   columns=["one", "two"])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [74]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"],
                   columns=["three", "four"])
df2

,three,four
a,5,6
c,7,8


In [75]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

Aquí el argumento keys se utiliza para crear un índice jerárquico donde el primer nivel puede utilizarse para identificar cada uno de los objetos DataFrame concatenados.

Si pasa un diccionario de objetos en lugar de una lista, se utilizarán las claves del diccionario para la opción keys:

In [76]:
pd.concat({"level1": df1, "level2": df2}, axis="columns")

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

Existen argumentos adicionales que regulan cómo se crea el índice jerárquico, ver tabla:

**Argumento**    |      **Descripción**

`objs`:  Lista o diccionario de objetos pandas a concatenar; este es el único argumento requerido.

`axis`: Eje sobre el que concatenar; por defecto se concatena por filas (axis="index")

`join`: Ya sea "interior" o "exterior" ("exterior" por defecto); si intersecar (interior) o unir (exterior) índices a lo largo de los otros ejes.

`keys`: Valores que se asocian a los objetos que se concatenan, formando un índice jerárquico a lo largo del eje de concatenación; puede ser una lista o array de valores arbitrarios, un array de tuplas o una lista de arrays (si se pasan arrays de varios niveles en niveles).

`levels`: Índices específicos que se utilizarán como nivel o niveles de índice jerárquico si se pasan claves.

`names`: Nombres de los niveles jerárquicos creados si se pasan claves y/o niveles

`verify_integrity`: Comprueba si hay duplicados en el nuevo eje del objeto concatenado y lanza una excepción si es así; por defecto (False) permite duplicados.

`ignore_index`: No preservar los índices a lo largo del eje de concatenación, en su lugar producir un nuevo índice `range(total_length)`

Por ejemplo, podemos nombrar los niveles de eje creados con el argumento `names`:

In [77]:
import pandas as pd
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"],
          names=["upper", "lower"])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

Una última consideración se refiere a los DataFrames en los que el índice de fila no contiene ningún dato relevante:

In [78]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)),
                   columns=["a", "b", "c", "d"])
df1

,a,b,c,d
0,1.248804,0.774191,-0.319657,-0.624964
1,1.078814,0.544647,0.855588,1.343268
2,-0.267175,1.793095,-0.652929,-1.886837


In [79]:
df2 = pd.DataFrame(np.random.standard_normal((2, 3)),
                   columns=["b", "d", "a"])
df2

,b,d,a
0,1.059626,0.644448,-0.007799
1,-0.449204,2.448963,0.667226


En este caso, puede pasar `ignore_index=True`, que descarta los índices de cada DataFrame y concatena los datos sólo en las columnas, asignando un nuevo índice por defecto:

In [80]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,1.248804,0.774191,-0.319657,-0.624964
1,1.078814,0.544647,0.855588,1.343268
2,-0.267175,1.793095,-0.652929,-1.886837
3,-0.007799,1.059626,NaN,0.644448
4,0.667226,-0.449204,NaN,2.448963


### Combinación de datos con solapamiento

Existe otra situación de combinación de datos que no puede expresarse como una operación de fusión (merge) o concatenación. Puede tener dos conjuntos de datos con índices que se solapan total o parcialmente. Como ejemplo , considere la función `where` de NumPy, que realiza el equivalente orientado en arrays de una expresión if-else:

In [81]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index=["f", "e", "d", "c", "b", "a"])
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [82]:
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index=["a", "b", "c", "d", "e", "f"])
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [83]:
np.where(pd.isna(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

Aquí, siempre que los valores en a sean nulos, se seleccionan los valores de b, de lo contrario se seleccionan los valores no nulos de a. El uso de `numpy.where` no comprueba si las etiquetas de índice están alineadas o no (y ni siquiera requiere que los objetos tengan la misma longitud), así que si quieres alinear valores por índice, utiliza el método `combine_first` de la serie:

In [84]:
a.combine_first(b)

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    5.0
dtype: float64

Con DataFrames, `combine_first` hace lo mismo columna por columna, por lo que se puede pensar en ello como "parchear" (patching) los datos que faltan en el objeto de llamada con los datos del objeto que se pasa:

In [85]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
                    "b": [np.nan, 2., np.nan, 6.],
                    "c": range(2, 18, 4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [86]:
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                    "b": [np.nan, 3., 4., 6., 8.]})
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [87]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


La salida de `combine_first` con objetos DataFrame tendrá la unión de todos los nombres de columna.

## 3.3 Remodelar (Reshaping) y pivotar (Pivoting)

Existen varias operaciones básicas para reorganizar datos tabulares. Se denominan operaciones de `reshape` o `pivot`.

### Remodelación (Reshaping) con indexación jerárquica

La indexación jerárquica proporciona una forma coherente de reorganizar los datos en un DataFrame. Existen dos acciones principales:

`stack`: Esto "gira" o pivota de las columnas de los datos a las filas.

`unstack`: Pivota de las filas a las columnas.

Se ilustrarán estas operaciones con una serie de ejemplos. Consideremos un pequeño DataFrame con arrays de cadenas como índices de fila y columna:

In [88]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"],
                    name="number"))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Utilizando el método `stack` en estos datos, las columnas pivotan en las filas, produciendo una Serie:

In [89]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

A partir de una Serie indexada jerárquicamente, puede reorganizar los datos de nuevo en un DataFrame con `unstack`:

In [90]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Por defecto, el nivel más interno está desapilado (igual que la pila). Puedes desapilar(unstack) un nivel diferente pasando un número o nombre de nivel:

In [91]:
result.unstack(level=0)


state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [92]:
result.unstack(level="state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


El desapilamiento puede introducir datos que faltan si no se encuentran todos los valores del nivel en cada subgrupo:

In [93]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")
s1

a    0
b    1
c    2
d    3
dtype: Int64

In [94]:
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")
s2

c    4
d    5
e    6
dtype: Int64

In [95]:
data2 = pd.concat([s1, s2], keys=["one", "two"])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

El apilamiento (Stacking) filtra por defecto los datos que faltan, por lo que la operación es más fácilmente invertible:

In [96]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [97]:
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [98]:
data2.unstack().stack(dropna=False)

one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

Cuando se desapila (unstack) en un DataFrame, el nivel desapilado se convierte en el nivel más bajo del resultado:

In [99]:
df = pd.DataFrame({"left": result, "right": result + 5},
                  columns=pd.Index(["left", "right"], name="side"))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [100]:
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

Al igual que con `unstack`, al llamar a `stack` podemos indicar el nombre del eje a apilar:

In [101]:
df.unstack(level="state").stack(level="side")

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

### Pasar del formato "largo" (Long) al "ancho (Wide)

Una forma habitual de almacenar múltiples series temporales en bases de datos y archivos CSV es lo que a veces se denomina formato largo o apilado (stacked format). En este formato, los valores individuales se representan mediante una única fila en una tabla, en lugar de múltiples valores por fila.

Carguemos algunos datos de ejemplo y hagamos una pequeña limpieza de series temporales y otros datos:

In [102]:
data = pd.read_csv("macrodata.csv")

In [103]:
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]

In [104]:
data.head()

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2


En primer lugar, se ha utilizado `pandas.PeriodIndex` (que representa intervalos de tiempo en lugar de puntos en el tiempo), lo veremos con más detalle en el tema de Series temporales, para combinar las columnas de `year` y `quarter` y establecer el índice para que consista en valores `datetime` al final de cada trimestre:

In [105]:
periods = pd.PeriodIndex(year=data.pop("year"),
                         quarter=data.pop("quarter"),
                         name="date")
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [106]:
data.index = periods.to_timestamp("D")
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


Aquí se ha utilizado el método `pop` en el DataFrame, que devuelve una columna al mismo tiempo que la elimina del DataFrame.

A continuación, se selecciona un subconjunto de columnas y se le da el nombre `"item"` al índice de columnas:

In [107]:
data = data.reindex(columns=["realgdp", "infl", "unemp"])
data.columns.name = "item"
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


Por último, se ha hecho un reshape con `stack`, convierte los nuevos niveles de índice en columnas con `reset_index` y, por último, se le da el nombre `"value"` a la columna que contiene los valores de los datos:

In [108]:
long_data = (data.stack()
             .reset_index()
             .rename(columns={0: "value"}))

Ahora, `ldata` se ve así:

In [109]:
long_data[:10]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


En este formato denominado largo para series temporales múltiples, cada fila de la tabla representa una única observación.

Los datos se almacenan con frecuencia de esta forma en bases de datos relacionales SQL, ya que un esquema fijo (nombres de columna y tipos de datos) permite que el número de valores distintos en la columna de `ítem` cambie a medida que se añaden datos a la tabla. 

En el ejemplo anterior, `date` y `item` suelen ser las claves primarias (en el lenguaje de las bases de datos relacionales), lo que ofrece integridad relacional y facilita las uniones (joins). En algunos casos, puede ser más difícil trabajar con los datos en este formato; es posible que prefiera tener un DataFrame que contenga una columna por cada valor de elemento distinto indexado por marcas de tiempo (timestamps) en la columna de fecha(date). El método `pivot` de DataFrame realiza exactamente esta transformación:

In [110]:
pivoted = long_data.pivot(index="date", columns="item",
                          values="value")
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


Los dos primeros valores pasados son las columnas que se utilizarán, respectivamente, como índice de fila y de columna, y finalmente una columna de valor opcional para rellenar el DataFrame. Supongamos que tiene dos columnas de valores que desea remodelar (reshape) simultáneamente:

In [111]:
long_data["value2"] = np.random.standard_normal(len(long_data))
long_data[:10]

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,0.802926
1,1959-01-01,infl,0.000,0.575721
2,1959-01-01,unemp,5.800,1.381918
3,1959-04-01,realgdp,2778.801,0.000992
4,1959-04-01,infl,2.340,-0.143492
5,1959-04-01,unemp,5.100,-0.206282
6,1959-07-01,realgdp,2775.488,-0.222392
7,1959-07-01,infl,2.740,-1.682403
8,1959-07-01,unemp,5.300,1.811659
9,1959-10-01,realgdp,2785.204,-0.351305


In [112]:
# long_data.index.name = None

Omitiendo el último argumento, se obtiene un DataFrame con columnas jerárquicas:

In [113]:
pivoted = long_data.pivot(index="date", columns="item")
pivoted.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.575721  0.802926  1.381918
1959-04-01  2.34  2778.801   5.1 -0.143492  0.000992 -0.206282
1959-07-01  2.74  2775.488   5.3 -1.682403 -0.222392  1.811659
1959-10-01  0.27  2785.204   5.6  0.128317 -0.351305 -1.313554
1960-01-01  2.31  2847.699   5.2 -0.615939  0.498327  0.174072

In [114]:
pivoted["value"].head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


Tenga en cuenta que pivotar es equivalente a crear un índice jerárquico utilizando `set_index` seguido de una llamada a `unstack`:

In [115]:
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")
unstacked.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.575721  0.802926  1.381918
1959-04-01  2.34  2778.801   5.1 -0.143492  0.000992 -0.206282
1959-07-01  2.74  2775.488   5.3 -1.682403 -0.222392  1.811659
1959-10-01  0.27  2785.204   5.6  0.128317 -0.351305 -1.313554
1960-01-01  2.31  2847.699   5.2 -0.615939  0.498327  0.174072

### Pasar (Pivoting) del formato "ancho" (Wide) al "largo" (Long)

Una operación inversa al pivote para DataFrames es `pandas.melt`. En lugar de transformar una columna en muchas en un nuevo DataFrame, fusiona(merges) múltiples columnas en una, produciendo un DataFrame más largo que el de entrada. Veamos un ejemplo:

In [116]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                   "A": [1, 2, 3],
                   "B": [4, 5, 6],
                   "C": [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


La columna "key" puede ser un indicador de grupo, y las otras columnas son valores de datos. Al utilizar `pandas.melt`, debemos indicar qué columnas (si las hay) son indicadores de grupo. Usemos aquí `"key"` como único indicador de grupo:

In [117]:
melted = pd.melt(df, id_vars="key")
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


Usando `pivot`, podemos volver (reshape) al diseño original:

In [118]:
reshaped = melted.pivot(index="key", columns="variable",
                        values="value")
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


Dado que el resultado de `pivot` crea un índice a partir de la columna utilizada como etiquetas de fila, es posible que deseemos utilizar `reset_index` para volver a mover los datos a una columna:

In [119]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


También puede especificar un subconjunto de columnas para utilizarlas como columnas de valores:

In [120]:
pd.melt(df, id_vars="key", value_vars=["A", "B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


`pandas.melt` también se puede utilizar sin ningún identificador de grupo:

In [121]:
pd.melt(df, value_vars=["A", "B", "C"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [122]:
pd.melt(df, value_vars=["key", "A", "B"])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
